In [1]:
import os
import gc
import numpy as np
import pandas as pd
from tqdm import tqdm
from typing import Dict, List, Tuple, NamedTuple, Callable
import scml
import mylib

In [2]:
percentiles=[.01, .05, .1, .2, .3, .4, .5, .6, .7, .8, .9, .95, .99]
pd.set_option("use_inf_as_na", True)
pd.set_option("max_info_columns", 9999)
pd.set_option("display.max_columns", 9999)
pd.set_option("display.max_rows", 9999)
pd.set_option('max_colwidth', 9999)
tqdm.pandas()

In [3]:
%%time
df = pd.read_csv("input/js18/train.csv", engine="c", low_memory=False)
tmp = pd.read_csv("input/js18/test.csv", engine="c", low_memory=False)
df = df.append(tmp, ignore_index=True)
df.rename(columns={"comment_text": "text"}, inplace=True)
cols = ["toxic", "severe_toxic", "obscene", "threat", "insult", "identity_hate"]
df.drop(columns=["id"] + cols, inplace=True)
df.drop_duplicates(["text"], keep='first', inplace=True, ignore_index=True)
#df[cols] = df[cols].astype(np.int8) 
#df["label_sum"] = df["toxic"] + df["severe_toxic"] + df["obscene"] + df["threat"] + df["insult"] + df["identity_hate"]
#df.drop(index=df[df.label_sum == 0].index, inplace=True)
#df.drop(columns=["label_sum"], inplace=True)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 312735 entries, 0 to 312734
Data columns (total 1 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   text    312735 non-null  object
dtypes: object(1)
memory usage: 2.4+ MB
Wall time: 2.07 s


In [4]:
df["length"] = df["text"].str.len()
df["length"].describe(percentiles=percentiles)

count    312735.000000
mean        379.773262
std         591.767791
min           1.000000
1%           20.000000
5%           30.000000
10%          42.000000
20%          71.000000
30%         105.000000
40%         145.000000
50%         193.000000
60%         260.000000
70%         350.000000
80%         502.000000
90%         848.000000
95%        1314.000000
99%        3487.000000
max        5000.000000
Name: length, dtype: float64

In [5]:
df = df[(df["length"] > 71) & (df["length"] < 502)].copy()
df.drop(columns=["length"], inplace=True)
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 186644 entries, 0 to 312734
Data columns (total 1 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   text    186644 non-null  object
dtypes: object(1)
memory usage: 2.8+ MB


In [6]:
scml.find_missing_values(df)

,Total,Percent,Type
text,0,0.0,object


# Preprocess Text
Speed dropped from 2400 to 600 it/s

In [7]:
def pre1(row) -> str:
    return mylib.pre1(row["text"])


def pre2(row) -> str:
    return mylib.pre2(row["text1"])


def pre3(row) -> str:
    return mylib.pre3(row["text2"])


col = "text1"
print(col)
df[col] = df.progress_apply(pre1, axis=1)
col = "text2"
print(col)
df[col] = df.progress_apply(pre2, axis=1)
col = "text3"
print(col)
df[col] = df.progress_apply(pre3, axis=1)

text1


100%|██████████████████████████████████████| 186644/186644 [06:06<00:00, 508.80it/s]


text2


100%|█████████████████████████████████████| 186644/186644 [2:13:19<00:00, 23.33it/s]


text3


100%|██████████████████████████████████████| 186644/186644 [30:20<00:00, 102.51it/s]


In [8]:
cols = ["text", "text1", "text2", "text3"]
df[cols].sample(20).head(20)

,text,text1,text2,text3
196252,""" \n\n == Korra Soundtrack for Book 1: Air announced! ) == \n\n About the lack of a Korra soundtrack.......this has changed! Last night,I found this entry on the Best Buy page under """"Korra"""" \n http://www.bestbuy.com/site/Legend+Of+Korra%3A+Original+Music+From+Book+One+-+CD/21563275.p?id=2717954&skuId;=21563275&st;=Korra&lp;=3&cp;=1 \n\n The 1st soundtrack comes out on July 16th,2013. Please change the music section of this article to reflect this. Thanks in advance! D """,""" == Korra Soundtrack for Book 1: Air announced!) == About the lack of a Korra soundtrack.......this has changed! Last night,I found this entry on the Best Buy page under """"Korra"""" The 1st soundtrack comes out on July 16th,2013. Please change the music section of this article to reflect this. Thanks in advance! D """,""" == Korra Soundtrack for Book 1: Air announced!) == About the lack of a Korra soundtrack...this has changed! Last night,I found this entry on the Best Buy page under """"Korra"""" The 1st soundtrack comes out on July 16th,2013. Please change the music section of this article to reflect this. Thanks in advance! D """,korra soundtrack for book 1 air announce about the lack of a korra soundtrack this have change last night i find this entry on the best buy page under korra the 1st soundtrack come out on july 16th 2013 please change the music section of this article to reflect this thank in advance d
106074,"I just looked at her official service photo (from when she was a 2LT: (File:2LT Tulsi Gabbard Tamayo.jpg) and reported what I saw there (it is very easy to recognize the German award). I didn't recognize the Kuwaiti medal, nor could I find it in a cursory search.","I just looked at her official service photo (from when she was a 2LT: (File:2LT Tulsi Gabbard Tamayo.jpg) and reported what I saw there (it is very easy to recognize the German award). I didn't recognize the Kuwaiti medal, nor could I find it in a cursory search.","I just looked at her official service photo (from when she was a 2LT: (File:2LT Tulsi Gabbard Tamayo.jpg) and reported what I saw there (it is very easy to recognize the German award). I did not recognize the Kuwaiti medal, nor could I find it in a cursory search.",i just look at her official service photo from when she be a 2lt file 2lt tulsi gabbard tamayo jpg and report what i see there it be very easy to recognize the german award i do not recognize the kuwaiti medal nor could i find it in a cursory search
16294,"""\ngot it. You think """"apostrophe"""" is a foreign word. Nope. It's English.""",""" got it. You think """"apostrophe"""" is a foreign word. Nope. It's English.""",""" got it. You think """"apostrophe"""" is a foreign word. [Nope; no]. it is English.""",get it you think apostrophe be a foreign word nope no it be english
152645,"""\nThat makes perfect sense. It's just the line """"The events of the game are set after those of the film Alien 3"""" that confused me. — (t – c) """,""" That makes perfect sense. It's just the line """"The events of the game are set after those of the film Alien 3"""" that confused me. - (t - c) """,""" That makes perfect sense. it is just the line """"The events of the game are set after those of the film Alien 3"""" that confused me. - (t - c) """,that make perfect sense it be just the line the event of the game be set after those of the film alien 3 that confuse i t c
149850,"Many thanks for warring but it is Wikipadida who is making Vandalism and posting the lies and unsupported quotation\n\nIf Wikipadia want to threat somebody, this is not only Vandalism, that is called differently, but in Syria, we are free and honest","Many thanks for warring but it is Wikipadida who is making Vandalism and posting the lies and unsupported quotation If Wikipadia want to threat somebody, this is not only Vandalism, that is called differently, but in Syria, we are free and honest","Many thanks for warring but is Wikipadida who is making Vandalism a

In [9]:
df[cols].sample(20).head(20)

,text,text1,text2,text3
41293,"I know that. That was Jarlaxle. I never said that you were jarlaxle, or that jarlaxle was you. I still don't see where you're getting that idea. | Talk |","I know that. That was Jarlaxle. I never said that you were jarlaxle, or that jarlaxle was you. I still don't see where you're getting that idea. | Talk |","I know that. That was Jarlaxle. I never said that you were jarlaxle, or that jarlaxle was you. I still do not see where you are getting that idea. | Talk |",i know that that be jarlaxle i never say that you be jarlaxle or that jarlaxle be you i still do not see where you be get that idea talk
241518,"::::::::The AfD made little mention of the community center, so I dont think its an entirely hasty decision. The worst thing that could happen would be that it was undone. Still, I'm OK with waiting if you'd prefer.—","The AfD made little mention of the community center, so I dont think its an entirely hasty decision. The worst thing that could happen would be that it was undone. Still, I'm OK with waiting if you'd prefer. -","The AfD made little mention of the community center, so I dont think its an entirely hasty decision. The worst thing that could happen would be that it was undone. Still, I am OK with waiting if you would prefer. -",the afd make little mention of the community center so i do nt think its an entirely hasty decision the bad thing that could happen would be that it be undo still i be ok with wait if you would prefer
69107,Fuck you 2.0 \n\nYou are an idiot and an admin abuser.\nI will keep coming back until my edits stay!,Fuck you 2.0 You are an idiot and an admin abuser. I will keep coming back until my edits stay!,Fuck you 2.0 You are an idiot and an admin abuser. I will keep coming back until my edits stay!,fuck you 2 0 you be an idiot and an admin abuser i will keep come back until my edit stay
15531,"""\nI'll just cut and paste what I wrote above then """"It quite clearly says Green Brigade on the green inverted banner beneath the the one that says """"WALK ALONE"""" when you look at the pictures on the reference given Here: http://www.dailymail.co.uk/sport/football/article-1363339/Celtic-2-Hamilton-0-Commons-double-fires-Hoops-points-clear-Rangers.html."""" """,""" I'll just cut and paste what I wrote above then """"It quite clearly says Green Brigade on the green inverted banner beneath the the one that says """"WALK ALONE"""" when you look at the pictures on the reference given Here: """,""" I will just cut and paste what I wrote above then """"It quite clearly says Green Brigade on the green inverted banner beneath the one that says """"WALK ALONE"""" when you look at the pictures on the reference given Here: """,i will just cut and paste what i write above then it quite clearly say green brigade on the green inverted banner beneath the one that say walk alone when you look at the picture on the reference give here
273321,"""I created this page to help alleviate the pressure created by Europeans looking for an English equivalent of the office of scabino/schoffe/schepen/echevin, etc. Hopefully an expert in the area will create a page similar to the page created for """"schepen"""". They were going to that page, thinking that it was the English word for the office. Even the Turks and the Chinese were misleadingly redirecting to the """"schepen page"""". (They still are.) ""","""I created this page to help alleviate the pressure created by Europeans looking for an English equivalent of the office of scabino/schoffe/schepen/echevin, etc. Hopefully an expert in the area will create a page similar to the page created for """"schepen""."" They were going to that page, thinking that it was the English word for the office. Even the Turks and the Chinese were misleadingly redirecting to the """"schepen page""."" (They still are.) ""","""I created this page to help alleviate the pressure created by Europeans looking for an English equivalent of the office of scabino/schoffe/sch

In [10]:
df[cols].sample(20).head(20)

,text,text1,text2,text3
226281,""" \n\n == Protected edit request on 28 Feb 2015 == \n\n \n\n Change the text of the redirect to """"State v. Zimmerman#Juror comments"""" in order to resolve a double redirect. All the best: , 19:21, 28 February 2015 (UTC). \n """,""" == Protected edit request on 28 Feb 2015 == Change the text of the redirect to """"State v. Zimmerman#Juror comments"""" in order to resolve a double redirect. All the best: , 19:21, 28 February 2015 (UTC). """,""" == Protected edit request on 28 Feb 2015 == Change the text of the redirect to """"State v. Zimmerman#Juror comments"""" in order to resolve a double redirect. All the best: , 19:21, 28 February 2015 (UTC). """,protect edit request on 28 feb 2015 change the text of the redirect to state v zimmerman juror comment in order to resolve a double redirect all the good 19 21 28 february 2015 utc
1642,wiki is non-commercial. so it is allowed to use maps of columbia university in wikipedia as sources and evidence. i just have got a positive answer from Columbia University for using of their maps. and sool i will do that. thank you for your message.,wiki is non-commercial. so it is allowed to use maps of columbia university in wikipedia as sources and evidence. i just have got a positive answer from Columbia University for using of their maps. and sool i will do that. thank you for your message.,[wiki; a website or database developed collaboratively by an online community] is non-commercial. so it is allowed to use maps of columbia university in wikipedia as sources and evidence. i just have got a positive answer from Columbia University for using of their maps. and sool i will do that. thank you for your message.,wiki a website or database develop collaboratively by an online community be non commercial so it be allow to use map of columbia university in wikipedia as source and evidence i just have get a positive answer from columbia university for use of their map and sool i will do that thank you for your message
290962,"Also, in case anyone wants to know, the thin, flexible membrane on the inside of the egg shell is also good for worms. They like to lay their eggs on it. I don't have a source for that, except my own personal experience.","Also, in case anyone wants to know, the thin, flexible membrane on the inside of the egg shell is also good for worms. They like to lay their eggs on it. I don't have a source for that, except my own personal experience.","Also, in case anyone wants to know, the thin, flexible membrane on the inside of the egg shell is also good for worms. They like to lay their eggs on it. I do not have a source for that, except my own personal experience.",also in case anyone want to know the thin flexible membrane on the inside of the egg shell be also good for worm they like to lay their egg on it i do not have a source for that except my own personal experience
306217,""":I'm not interpreting what they have filed for. I am referencing other sources which do the interpretation. This is considered legitimate by Wikipedia standards. The entire text of the lawsuit isn't publically available, so this is the best we can do. \n :Just because no source uses the exact words """"based on"""" doesn't mean that it's """"interpretation"""" of my own to say that the suit concerns whether Smallville is based on Superboy. \n\n """,""":I'm not interpreting what they have filed for. I am referencing other sources which do the interpretation. This is considered legitimate by Wikipedia standards. The entire text of the lawsuit isn't publically available, so this is the best we can do.:Just because no source uses the exact words """"based on"""" doesn't mean that it's """"interpretation"""" of my own to say that the suit concerns whether Smallville is based on Superboy. """,""":I am not interpreting what they have filed for. I am referencing other sources which do the interpretation. This considered legitimate by Wikipedia standards. The entire text of the 

# Review data

In [11]:
col = "worker"
df[col] = np.random.randint(1, 128, size=len(df), dtype=np.int8)
cols = ["worker", "text", "text1", "text2", "text3"]
df[cols].info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 186644 entries, 0 to 312734
Data columns (total 5 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   worker  186644 non-null  int8  
 1   text    186644 non-null  object
 2   text1   186644 non-null  object
 3   text2   186644 non-null  object
 4   text3   186644 non-null  object
dtypes: int8(1), object(4)
memory usage: 7.3+ MB


In [12]:
%%time
df[cols].to_parquet("output/pre_js18.parquet", index=False)

Wall time: 825 ms
